In [1]:
import requests
import os
import time

# Main Function to download images from iNaturalist API with rate limiting
def download_images(taxon_id, user_id, output_dir, max_requests_per_minute=100):
    os.makedirs(output_dir, exist_ok=True)
    url = 'https://api.inaturalist.org/v1/observations'
    params = {
        'taxon_id': taxon_id,  # Taxon ID for the species
        'per_page': 200,  # Number of images per page to download
        'user_id': user_id,  # User ID for the user
        'photos': True  # Only include observations with photos
    }
    
    # Calculate the delay needed to stay within rate limits
    delay = 60 / max_requests_per_minute
    
    response = requests.get(url, params=params)
    data = response.json()

    image_urls = []
    for result in data['results']:
        for photo in result.get('photos', []):
            # Check for the largest available image
            img_url = photo['url'].replace('square', 'original')
            image_urls.append(img_url)

    for i, img_url in enumerate(image_urls):
        try:
            # Handle relative URLs
            if not img_url.startswith('http'):
                img_url = 'https:' + img_url
            
            # Request the image data and write to file
            img_data = requests.get(img_url).content
            image_name = f'{user_id}_{taxon_id}_image_{i + 1}.jpg'
            with open(os.path.join(output_dir, image_name), 'wb') as img_file: 
                img_file.write(img_data)
            print(f'Downloaded {img_url} as {image_name}.jpg')
        except Exception as e:
            print(f'Failed to download {img_url}: {e}')
        
        # Sleep to ensure rate limit is not exceeded
        if (i + 1) % max_requests_per_minute == 0:
            print(f'Rate limit reached. Waiting for 60 seconds.')
            time.sleep(60)
        else:
            time.sleep(delay)

In [5]:
#Loading the images

# Prompt the user for input values
taxon_id = input("Enter the taxon ID of the species you want to download images for: ")
user_id = input("Enter the user ID of the user: ")
output_dir = input("Enter the folder name where you want to save the images: ")

# Call the download_images function with the user inputs
download_images(taxon_id=int(taxon_id), output_dir=output_dir, user_id=user_id)

Downloaded https://inaturalist-open-data.s3.amazonaws.com/photos/180010577/original.jpeg as sec_research_42051_image_1.jpg.jpg
Downloaded https://inaturalist-open-data.s3.amazonaws.com/photos/180009446/original.jpeg as sec_research_42051_image_2.jpg.jpg
Downloaded https://inaturalist-open-data.s3.amazonaws.com/photos/180009434/original.jpeg as sec_research_42051_image_3.jpg.jpg
Downloaded https://inaturalist-open-data.s3.amazonaws.com/photos/180009291/original.jpeg as sec_research_42051_image_4.jpg.jpg
Downloaded https://inaturalist-open-data.s3.amazonaws.com/photos/180008917/original.jpeg as sec_research_42051_image_5.jpg.jpg
Downloaded https://inaturalist-open-data.s3.amazonaws.com/photos/83185014/original.png as sec_research_42051_image_6.jpg.jpg
Downloaded https://inaturalist-open-data.s3.amazonaws.com/photos/83184548/original.png as sec_research_42051_image_7.jpg.jpg
Downloaded https://inaturalist-open-data.s3.amazonaws.com/photos/67955787/original.png as sec_research_42051_image_

KeyboardInterrupt: 